<html>
    <h1>RESTAURANT RECOMMENDATION</h1>
</html>

<html>
    <h3>Import the needed Libraries</h3>
</html>

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

<html>
    <h3> Download the dataset and save it as newyork_data</h3>
</html>

In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


<html>
<h3>Load and explore the data</h3>
</html>

In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [5]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

<html>
<h3>Features key is transformed into the Dataset</h3>
</html>

In [7]:
neighborhoods_data = newyork_data['features']

In [8]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

<html>
<h3>Transform the data into pandas dataframe</h3>
</html>

In [10]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

In [11]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


<html>
<h3>Filling the dataframe one row at a time</h3>
</html>

In [12]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [13]:
neighborhoods.head(10)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [14]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


<html>
<h3>Geopy library is used to get latitude and longitude of NewYork</h3>
</html>

In [15]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


<html>
<h3>Map of New York</h3>
</html>

In [16]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)
    
    
map_newyork

<html>
<h3>New Dataframe for Manhattan Data</h3>
</html>

In [17]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


<html>
<h3>Geopy library is used to get latitude and longitude of Manhattan</h3>
</html>

In [18]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


<html>
<h3>Map of Manhattan</h3>
</html>

In [60]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

<html>
<h3>Foursquare Credentials</h3>
</html>

In [59]:
#This is removed because of the sensitive information

CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version



<html>
<h3>Exploring the first neighborhood</h3>
</html>

In [21]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [22]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] 
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] 

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


<html>
<h3>Retrieving venues within a radius of 5000</h3>
</html>

In [24]:
LIMIT = 1000 

radius = 5000 


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 


'https://api.foursquare.com/v2/venues/explore?&client_id=EOPZE5Y2TZXU5IENQJGKWQFSOW22PNA0WYYTKZVWYOOUTGHH&client_secret=GZMF0K53ZLM2DURO0YR0MHCVFF30AR5A1QOYJ1RF52VNGSW0&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=5000&limit=1000'

In [25]:
results = requests.get(url).json()

<html>
<h3>Extracting categories of venues</h3>
</html>

In [26]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) 
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Bikram Yoga,Yoga Studio,40.876844,-73.906204
1,Sam's Pizza,Pizza Place,40.879435,-73.905859
2,Garden Gourmet Market,Gourmet Shop,40.881350,-73.903389
3,Indian Road Café,Café,40.872922,-73.918459
4,Inwood Hill Park,Park,40.870858,-73.920129


In [28]:
pd.unique(nearby_venues['categories'])

array(['Yoga Studio', 'Pizza Place', 'Gourmet Shop', 'Café', 'Park',
       'Diner', 'Caribbean Restaurant', 'Pub', 'Bakery', 'Farmers Market',
       'Athletics & Sports', 'Mexican Restaurant', 'Japanese Restaurant',
       'Scenic Lookout', 'Latin American Restaurant', 'Tapas Restaurant',
       'Beer Bar', 'Wine Shop', 'Deli / Bodega', 'Burger Joint',
       'Wine Bar', 'Bar', 'Coffee Shop', 'Pet Store', 'Track',
       'History Museum', 'Bistro', 'Plaza', 'Trail', 'Museum',
       'Warehouse Store', 'Venezuelan Restaurant', 'Spanish Restaurant',
       'Dog Run', 'Basketball Court', 'Veterinarian', 'Garden',
       'American Restaurant', 'Mediterranean Restaurant', 'Liquor Store',
       'Botanical Garden', 'Beer Garden', 'Juice Bar',
       'Italian Restaurant', 'Steakhouse', 'Food & Drink Shop',
       'Sushi Restaurant', 'Market', 'Martial Arts Dojo',
       'Ice Cream Shop', 'Stables', 'Exhibit', 'Restaurant', 'Gift Shop'],
      dtype=object)

In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )



Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [32]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3088, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [33]:
pd.unique(manhattan_venues['Venue Category'])

array(['Pizza Place', 'Yoga Studio', 'Diner', 'Coffee Shop', 'Donut Shop',
       'Pharmacy', 'Tennis Stadium', 'Seafood Restaurant', 'Gym',
       'Supplement Shop', 'Steakhouse', 'Department Store',
       'American Restaurant', 'Sandwich Place', 'Ice Cream Shop',
       'Discount Store', 'Video Game Store', 'Bank', 'Kids Store',
       'Miscellaneous Shop', 'Shopping Mall', 'Deli / Bodega',
       'Greek Restaurant', 'Hotel', 'Spa', 'Bakery', 'Cocktail Bar',
       'Museum', 'English Restaurant', 'Tea Room', 'Hotpot Restaurant',
       'New American Restaurant', 'Indie Movie Theater', 'Roof Deck',
       'Spanish Restaurant', 'Chinese Restaurant', 'Noodle House',
       'Salon / Barbershop', 'Bike Shop', 'Historic Site',
       'Asian Restaurant', 'Boutique', 'Thai Restaurant', 'Music Venue',
       'Cosmetics Shop', 'Organic Grocery', 'Bubble Tea Shop',
       'Malay Restaurant', 'Supermarket', 'Snack Place',
       'Vietnamese Restaurant', 'Jewelry Store',
       'Paper / Office S

<html>
<h3>Retrieving only restaurants from venues</h3>
</html>

In [35]:
manhattan_restaurants_venues = manhattan_venues.loc[manhattan_venues['Venue Category'].str.split(' ').str[1] == 'Restaurant']

In [36]:
manhattan_restaurants_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
7,Marble Hill,40.876551,-73.910660,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant
14,Marble Hill,40.876551,-73.910660,Boston Market,40.877430,-73.905412,American Restaurant
27,Chinatown,40.715618,-73.994279,Kiki's,40.714476,-73.992036,Greek Restaurant
34,Chinatown,40.715618,-73.994279,The Fat Radish,40.715323,-73.991950,English Restaurant
36,Chinatown,40.715618,-73.994279,Da Yu Hot Pot 大渝火锅,40.716735,-73.995752,Hotpot Restaurant


In [37]:
manhattan_restaurants_venues = manhattan_restaurants_venues.groupby('Neighborhood').filter(lambda x : x['Venue Category'].count() >= 8)

In [38]:
manhattan_restaurants_venues.groupby(['Neighborhood']).count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Carnegie Hill,14,14,14,14,14,14
Central Harlem,13,13,13,13,13,13
Chelsea,13,13,13,13,13,13
Chinatown,33,33,33,33,33,33
Civic Center,20,20,20,20,20,20
Clinton,17,17,17,17,17,17
East Harlem,11,11,11,11,11,11
East Village,33,33,33,33,33,33
Financial District,19,19,19,19,19,19


In [39]:
print('There are {} uniques categories.'.format(len(manhattan_restaurants_venues['Venue Category'].unique())))

There are 61 uniques categories.


<html>
<h3>Performing One hot encoding</h3>
</html>

In [41]:
manhattan_onehot = pd.get_dummies(manhattan_restaurants_venues[['Venue Category']], prefix = "", prefix_sep="")
manhattan_onehot ['Neighborhood'] = manhattan_restaurants_venues['Neighborhood']
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Brazilian Restaurant,Cambodian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Cuban Restaurant,Czech Restaurant,Dumpling Restaurant,Egyptian Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hotpot Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Moroccan Restaurant,Paella Restaurant,Persian Restaurant,Peruvian Restaurant,Ramen Restaurant,Russian Restaurant,Scandinavian Restaurant,Seafood Restaurant,Shanghai Restaurant,Soba Restaurant,Spanish Restaurant,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Udon Restaurant,Vietnamese Restaurant
27,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
34,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
36,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
42,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
43,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [42]:
manhattan_onehot.shape

(716, 62)

In [43]:
manhattan_restaurant_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_restaurant_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Brazilian Restaurant,Cambodian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Cuban Restaurant,Czech Restaurant,Dumpling Restaurant,Egyptian Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hotpot Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Moroccan Restaurant,Paella Restaurant,Persian Restaurant,Peruvian Restaurant,Ramen Restaurant,Russian Restaurant,Scandinavian Restaurant,Seafood Restaurant,Shanghai Restaurant,Soba Restaurant,Spanish Restaurant,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Udon Restaurant,Vietnamese Restaurant
0,Carnegie Hill,0.000000,0.000000,0.071429,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.214286,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.071429
1,Central Harlem,0.000000,0.230769,0.153846,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
2,Chelsea,0.000000,0.000000,0.230769,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.153846,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.076923,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.153846,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
3,Chinatown,0.000000,0.000000,0.090909,0.000000,0.000000,0.060606,0.030303,0.030303,0.000000,0.000000,0.030303,0.000000,0.000000,0.181818,0.000000,0.000000,0.030303,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.060606,0.000000,0.000000,0.030303,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.060606,0.000000,0.060606,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060606,0.000000,0.030303,0.000000,0.000000,0.000000,0.030303,0.000000,0.030303,0.000000,0.00000,0.090909
4,Civic Center,0.000000,0.000000,0.250000,0.000000,0.000000,0.050000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.050000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.050000
5,Clint

In [44]:
manhattan_restaurant_grouped.shape

(36, 62)

<html>
<h3>Retrieving Top 5 venues of each neighborhood</h3>
</html>

In [45]:
num_top_venues = 5

for hood in manhattan_restaurant_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_restaurant_grouped[manhattan_restaurant_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Carnegie Hill----
                 venue  freq
0  Japanese Restaurant  0.21
1   Italian Restaurant  0.14
2    French Restaurant  0.14
3     Sushi Restaurant  0.07
4    Kosher Restaurant  0.07


----Central Harlem----
                 venue  freq
0   African Restaurant  0.23
1   Seafood Restaurant  0.15
2  American Restaurant  0.15
3   Chinese Restaurant  0.15
4    French Restaurant  0.15


----Chelsea----
                 venue  freq
0  American Restaurant  0.23
1   Italian Restaurant  0.15
2   Seafood Restaurant  0.15
3     Sushi Restaurant  0.08
4  Japanese Restaurant  0.08


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.18
1  Vietnamese Restaurant  0.09
2    American Restaurant  0.09
3       Malay Restaurant  0.06
4     Mexican Restaurant  0.06


----Civic Center----
                   venue  freq
0    American Restaurant  0.25
1      French Restaurant  0.25
2       Sushi Restaurant  0.10
3  Vietnamese Restaurant  0.05
4      Indian Restaurant  0.0

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<html>
<h3>Retrieving Top 10 venues</h3>
</html>

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_restaurant_grouped['Neighborhood']

for ind in np.arange(manhattan_restaurant_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_restaurant_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Carnegie Hill,Japanese Restaurant,French Restaurant,Italian Restaurant,Vietnamese Restaurant,Ramen Restaurant,Kosher Restaurant,Sushi Restaurant,Mexican Restaurant,Argentinian Restaurant,American Restaurant
1,Central Harlem,African Restaurant,Chinese Restaurant,Seafood Restaurant,French Restaurant,American Restaurant,Ethiopian Restaurant,Caribbean Restaurant,Himalayan Restaurant,Hawaiian Restaurant,Greek Restaurant
2,Chelsea,American Restaurant,Seafood Restaurant,Italian Restaurant,Chinese Restaurant,Sushi Restaurant,Mediterranean Restaurant,Ramen Restaurant,Mexican Restaurant,Japanese Restaurant,Czech Restaurant
3,Chinatown,Chinese Restaurant,Vietnamese Restaurant,American Restaurant,Malay Restaurant,Mexican Restaurant,Greek Restaurant,Asian Restaurant,Shanghai Restaurant,Korean Restaurant,English Restaurant
4,Civic Center,American Restaurant,French Restaurant,Sushi Restaurant,Vietnamese Restaurant,Italian Restaurant,Korean Restaurant,Falafel Restaurant,Cuban Restaurant,Australian Restaurant,Asian Restaurant


<html>
<h3>Performing K-Means clustering</h3>
</html>

In [48]:
kclusters = 5
manhattan_grouped_clustering = manhattan_restaurant_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)
kmeans.labels_[0:] 

array([1, 2, 2, 1, 1, 0, 4, 1, 2, 0, 0, 0, 1, 0, 2, 0, 0, 1, 1, 1, 2, 1,
       3, 2, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0], dtype=int32)

In [49]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
manhattan_merged.tail()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,Manhattan,Turtle Bay,40.752042,-73.967708,1.0,Sushi Restaurant,Seafood Restaurant,French Restaurant,Italian Restaurant,Greek Restaurant,Turkish Restaurant,American Restaurant,Japanese Restaurant,Asian Restaurant,Indian Restaurant
36,Manhattan,Tudor City,40.746917,-73.971219,1.0,Mexican Restaurant,Sushi Restaurant,Vietnamese Restaurant,Thai Restaurant,Asian Restaurant,Greek Restaurant,Seafood Restaurant,Shanghai Restaurant,French Restaurant,Japanese Restaurant
37,Manhattan,Stuyvesant Town,40.731000,-73.974052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,Manhattan,Flatiron,40.739673,-73.990947,0.0,Italian Restaurant,Mediterranean Restaurant,Japanese Restaurant,American Restaurant,Russian Restaurant,Kebab Restaurant,Sushi Restaurant,Cuban Restaurant,Thai Restaurant,Filipino Restaurant
39,Manhattan,Hudson Yards,40.756658,-74.000111,0.0,Italian Restaurant,American Restaurant,Peruvian Restaurant,Caucasian Restaurant,Spanish Restaurant,Thai Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Cuban Restaurant


In [50]:
manhattan_merged = manhattan_merged.dropna()
manhattan_merged['Cluster Labels'] = manhattan_merged['Cluster Labels'].astype(int)

In [51]:
manhattan_merged.tail()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Manhattan,Sutton Place,40.760280,-73.963556,0,Italian Restaurant,Mexican Restaurant,Greek Restaurant,Asian Restaurant,Mediterranean Restaurant,Chinese Restaurant,Persian Restaurant,Cambodian Restaurant,Indian Restaurant,Seafood Restaurant
35,Manhattan,Turtle Bay,40.752042,-73.967708,1,Sushi Restaurant,Seafood Restaurant,French Restaurant,Italian Restaurant,Greek Restaurant,Turkish Restaurant,American Restaurant,Japanese Restaurant,Asian Restaurant,Indian Restaurant
36,Manhattan,Tudor City,40.746917,-73.971219,1,Mexican Restaurant,Sushi Restaurant,Vietnamese Restaurant,Thai Restaurant,Asian Restaurant,Greek Restaurant,Seafood Restaurant,Shanghai Restaurant,French Restaurant,Japanese Restaurant
38,Manhattan,Flatiron,40.739673,-73.990947,0,Italian Restaurant,Mediterranean Restaurant,Japanese Restaurant,American Restaurant,Russian Restaurant,Kebab Restaurant,Sushi Restaurant,Cuban Restaurant,Thai Restaurant,Filipino Restaurant
39,Manhattan,Hudson Yards,40.756658,-74.000111,0,Italian Restaurant,American Restaurant,Peruvian Restaurant,Caucasian Restaurant,Spanish Restaurant,Thai Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Cuban Restaurant


<html>
<h3>Creating a cluster map</h3>
</html>

In [53]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<html>
<h3>Cluster Labels</h3>
</html>

In [54]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Italian Restaurant,French Restaurant,American Restaurant,Sushi Restaurant,Mexican Restaurant,Mediterranean Restaurant,Chinese Restaurant,Thai Restaurant,Tapas Restaurant,Seafood Restaurant
9,Yorkville,Italian Restaurant,Sushi Restaurant,Japanese Restaurant,Mexican Restaurant,Chinese Restaurant,Vietnamese Restaurant,Peruvian Restaurant,French Restaurant,Indian Restaurant,German Restaurant
10,Lenox Hill,Italian Restaurant,Sushi Restaurant,Turkish Restaurant,Thai Restaurant,Afghan Restaurant,Czech Restaurant,Cuban Restaurant,Chinese Restaurant,Greek Restaurant,Japanese Restaurant
13,Lincoln Square,Italian Restaurant,American Restaurant,French Restaurant,Mediterranean Restaurant,Greek Restaurant,Chinese Restaurant,Mexican Restaurant,Seafood Restaurant,Vietnamese Restaurant,Empanada Restaurant
14,Clinton,Italian Restaurant,Thai Restaurant,American Restaurant,Peruvian Restaurant,Brazilian Restaurant,Chinese Restaurant,Caucasian Restaurant,Seafood Restaurant,Korean Restaurant,Mediterranean Restaurant
18,Greenwich Village,Italian Restaurant,Sushi Restaurant,Vietnamese Restaurant,Indian Restaurant,Mediterranean Restaurant,Chinese Restaurant,Caribbean Restaurant,Tapas Restaurant,Ramen Restaurant,Falafel Restaurant
21,Tribeca,Italian Restaurant,American Restaurant,French Restaurant,Greek Restaurant,Sushi Restaurant,Seafood Restaurant,Korean Restaurant,Chinese Restaurant,Argentinian Restaurant,Indian Restaurant
23,Soho,Italian Restaurant,Mediterranean Restaurant,Sushi Restaurant,Egyptian Restaurant,Seafood Restaurant,Falafel Restaurant,Mexican Restaurant,Australian Restaurant,French Restaurant,American Restaurant
24,West Village,Italian Restaurant,American Restaurant,French Restaurant,Chinese Restaurant,Brazilian Restaurant,Mediterranean Restaurant,Korean Restaurant,Seafood Restaurant,Falafel Restaurant,Austrian Restaurant
27,Gramercy,Mexican Restaurant,Thai Restaurant,American Restaurant,Italian Restaurant,Sushi Restaurant,Filipino Restaurant,Ethiopian Restaurant,Egyptian Restaurant,Empanada Restaurant,English Restaurant


In [55]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Vietnamese Restaurant,American Restaurant,Malay Restaurant,Mexican Restaurant,Greek Restaurant,Asian Restaurant,Shanghai Restaurant,Korean Restaurant,English Restaurant
2,Washington Heights,Chinese Restaurant,Tapas Restaurant,Mexican Restaurant,Spanish Restaurant,Thai Restaurant,American Restaurant,Arepa Restaurant,Sushi Restaurant,Indian Restaurant,Seafood Restaurant
4,Hamilton Heights,Mexican Restaurant,Indian Restaurant,Chinese Restaurant,Sushi Restaurant,Caribbean Restaurant,Seafood Restaurant,Mediterranean Restaurant,Italian Restaurant,Japanese Restaurant,Egyptian Restaurant
12,Upper West Side,Italian Restaurant,Seafood Restaurant,Indian Restaurant,American Restaurant,Mediterranean Restaurant,Sushi Restaurant,Israeli Restaurant,Japanese Restaurant,Kosher Restaurant,Mexican Restaurant
15,Midtown,Japanese Restaurant,Sushi Restaurant,Cuban Restaurant,American Restaurant,Mediterranean Restaurant,Indian Restaurant,Szechuan Restaurant,French Restaurant,Turkish Restaurant,Chinese Restaurant
16,Murray Hill,Japanese Restaurant,Chinese Restaurant,American Restaurant,Cuban Restaurant,Mediterranean Restaurant,Hawaiian Restaurant,Italian Restaurant,Mexican Restaurant,Turkish Restaurant,French Restaurant
19,East Village,Mexican Restaurant,Ramen Restaurant,Vietnamese Restaurant,Seafood Restaurant,Greek Restaurant,Italian Restaurant,Japanese Restaurant,Filipino Restaurant,Sushi Restaurant,Moroccan Restaurant
20,Lower East Side,Chinese Restaurant,Vietnamese Restaurant,Japanese Restaurant,Ramen Restaurant,Mediterranean Restaurant,French Restaurant,Filipino Restaurant,Argentinian Restaurant,Italian Restaurant,American Restaurant
22,Little Italy,Chinese Restaurant,Italian Restaurant,Mediterranean Restaurant,Thai Restaurant,Seafood Restaurant,Japanese Restaurant,Hotpot Restaurant,French Restaurant,Falafel Restaurant,Egyptian Restaurant
25,Manhattan Valley,Mexican Restaurant,Vietnamese Restaurant,Caribbean Restaurant,Italian Restaurant,Japanese Restaurant,Ethiopian Restaurant,Korean Restaurant,Hawaiian Restaurant,Peruvian Restaurant,Chinese Restaurant


In [56]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Inwood,Mexican Restaurant,Chinese Restaurant,Caribbean Restaurant,Spanish Restaurant,American Restaurant,Seafood Restaurant,Empanada Restaurant,Greek Restaurant,German Restaurant,French Restaurant
5,Manhattanville,Seafood Restaurant,Mexican Restaurant,Italian Restaurant,Spanish Restaurant,Cuban Restaurant,Chinese Restaurant,Dumpling Restaurant,Ramen Restaurant,Falafel Restaurant,Sushi Restaurant
6,Central Harlem,African Restaurant,Chinese Restaurant,Seafood Restaurant,French Restaurant,American Restaurant,Ethiopian Restaurant,Caribbean Restaurant,Himalayan Restaurant,Hawaiian Restaurant,Greek Restaurant
17,Chelsea,American Restaurant,Seafood Restaurant,Italian Restaurant,Chinese Restaurant,Sushi Restaurant,Mediterranean Restaurant,Ramen Restaurant,Mexican Restaurant,Japanese Restaurant,Czech Restaurant
26,Morningside Heights,American Restaurant,Indian Restaurant,Greek Restaurant,Mexican Restaurant,Mediterranean Restaurant,Seafood Restaurant,Ethiopian Restaurant,English Restaurant,Egyptian Restaurant,Empanada Restaurant
29,Financial District,American Restaurant,Mexican Restaurant,Italian Restaurant,Falafel Restaurant,Japanese Restaurant,Mediterranean Restaurant,French Restaurant,Seafood Restaurant,Vietnamese Restaurant,English Restaurant


In [57]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Midtown South,Korean Restaurant,American Restaurant,Japanese Restaurant,Cuban Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Caribbean Restaurant,Persian Restaurant,Falafel Restaurant,Egyptian Restaurant


In [58]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,East Harlem,Mexican Restaurant,Thai Restaurant,French Restaurant,Seafood Restaurant,Cuban Restaurant,Vietnamese Restaurant,English Restaurant,Dumpling Restaurant,Egyptian Restaurant,Empanada Restaurant
